In [ ]:
exec("from functools import namedtuple\nfrom itertools import chain\nCacheEntry = namedtuple('CacheEntry', ['value', 'version'])\n\nclass CG:\n    def __init__(self):\n        self.nodes = dict()\n        self.nodes_cache = dict()\n        self.timestamp = 0\n\n    def compute_node_(self, name, args, kwargs, need_version=False):\n        max_version = float('-inf')\n\n        value_args = []\n        for a in args:\n            value, version = a(need_version=True)\n            value_args.append(value)\n            max_version = max(max_version, version)\n\n        value_kwargs = dict()\n        for k,v in kwargs.items():\n            value, version = v(need_version=True)\n            value_kwargs[k] = value\n            max_version = max(max_version, version)\n\n        if name not in self.nodes_cache or self.nodes_cache[name].version <= max_version:\n            print('cg: computing [{}]'.format(name))\n            self.nodes_cache[name] = CacheEntry(value=self.nodes[name](*value_args, **value_kwargs), version=self.timestamp)\n            self.timestamp += 1\n\n        if need_version:\n            return self.nodes_cache[name]\n        else:\n            return self.nodes_cache[name].value\n\n        \n    def node(self, *args, name=None, **kwargs):\n        def add_node(f):\n            name_ = name or f.__name__\n                        \n            self.nodes[name_] = f\n            if name_ in self.nodes_cache:\n                del self.nodes_cache[name_]\n                \n            def compute_node(need_version=False):\n                return self.compute_node_(name_, args, kwargs, need_version=need_version)\n                \n            return compute_node\n        return add_node\n\n    def vnode(self, *args, name=None, **kwargs):\n        def add_vnode(f):\n            def compute_vnode(*args_, **kwargs_):\n                dim = None\n                for a in chain(args_, kwargs_.values()):\n                    if isinstance(a, list):\n                        assert dim is None or dim == len(a), 'vector arguments should have same lengths'\n                        dim = len(a)\n                \n                assert dim is not None, 'vector operation should operate on at least one vector'\n                \n                res = []\n                for i in range(dim):\n                    args_i = []\n                    for a in args_:\n                        if isinstance(a, list):\n                            args_i.append(a[i])\n                        else:\n                            args_i.append(a)\n                            \n                    kwargs_i = dict()\n                    for k,v in kwargs_:\n                        if isinstance(v, list):\n                            kwargs_i[k] = v[i]\n                        else:\n                            kwargs_i[k] = v\n                    \n                    res.append(f(*args_i, **kwargs_i))\n                    \n                return res\n            \n            return self.node(*args, name=name or f.__name__, **kwargs)(compute_vnode)\n        return add_vnode\n                \n            \ncg = CG()")
exec('from functools import namedtuple\nfrom functools import partial as p\nfrom functools import reduce as r\n\ndef c(*fs): return p(r, (lambda x, f: f(x)), fs[::-1])\ndef lmap(f, a): return list(map(f, a))\ndef lfilter(f, a): return list(filter(f, a))\ndef nth(n): return (lambda v: v[n])\ndef div(d): return (lambda x: x/d)\ndef fst(v): return next(iter(v))\nsnd = nth(-1)\n\ndef infrange(start=0):\n    i = start\n    while 1:\n        yield i\n        i += 1\n        \ndef srange(n): return pd.Series(range(n), index=range(n))')

In [ ]:
import numpy as np
import torch as tc
import pandas as pd

import scipy.stats as stats
import statsmodels.api as sm

from tqdm import tqdm as tqdm

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
@cg.node()
def features():
    return pd.read_csv('../input/lish-moa/train_features.csv').set_index('sig_id').sort_index()
    
@cg.node()
def target():
    return pd.read_csv('../input/lish-moa/train_targets_scored.csv').set_index('sig_id').sort_index()

assert features().index.duplicated().sum() == 0
assert target().index.duplicated().sum() == 0

assert all(features().index == target().index)

In [ ]:
def prefix_filter(prefix, a): return lfilter(lambda s: s.startswith(prefix), features().columns)

@cg.node()
def cp_columns():
    return prefix_filter('cp_', features().columns)

@cg.node()
def g_columns(): 
    return prefix_filter('g-', features().columns)

@cg.node()
def c_columns():
    return prefix_filter('c-', features().columns)

assert sum(map(len, [cp_columns(), g_columns(), c_columns()])) == len(features().columns)

In [ ]:
@cg.node()
def train_mask():
    np.random.seed(21)
    return np.random.randint(2, size=len(features())).astype(bool) & (features().cp_type != 'ctl_vehicle')

@cg.node()
def test_mask():
    return ~train_mask() & (features().cp_type != 'ctl_vehicle')

@cg.node()
def train_features():
    return features().loc[train_mask()]

@cg.node()
def train_target():
    return target().loc[train_mask()]

@cg.node()
def test_features():
    return features().loc[test_mask()]

@cg.node()
def test_target():
    return target().loc[test_mask()]

In [ ]:
@cg.node()
def submit_train_mask():
    return features().cp_type != 'ctl_vehicle'

@cg.node()
def submit_train_features():
    return features()[submit_train_mask()]

@cg.node()
def submit_train_target():
    return target()[submit_train_mask()]

@cg.node()
def submit_test_features():
    return pd.read_csv('../input/lish-moa/test_features.csv').set_index('sig_id').sort_index()

assert submit_test_features().index.duplicated().sum() == 0

In [ ]:
def score(prob, true):
    prob = np.clip(prob, 1e-15, 1-1e-15)
    return -np.stack([
        np.log(1-prob),
        np.log(prob)
    ]).reshape(2, -1)[true.reshape(-1), np.arange(np.prod(true.shape))].mean()

In [ ]:
@cg.node()
def num_columns():
    return c_columns() + g_columns()

In [ ]:
@cg.node()
def local_pca_model():
    model = dict()
    for col in tqdm(train_target().columns):
        mask = train_target()[col] == 1
        model[col] = train_features()[mask]
    return model

In [ ]:
len(local_pca_model())

In [ ]:
from itertools import product
from scipy.spatial.distance import cdist

In [ ]:
@cg.node()
def test_local_pca_dist():
    dist = pd.DataFrame(index=test_features().index, columns=train_target().columns)
    
    for col in tqdm(train_target().columns):
        if len(local_pca_model()[col]) == 0:
            dist[col] = float('inf')
        else:
            dist[col] = cdist(test_features()[num_columns()].values, local_pca_model()[col][num_columns()]).min(axis=1)
        
    return dist

In [ ]:
test_local_pca_dist()

In [ ]:
@cg.node()
def get_best_alphas():
    best_alphas = []
    alphas = np.linspace(1e-2, 1, 100)
    for col in tqdm(train_target().columns):
        scores = [score(to_prob(test_local_pca_dist()[col].values.reshape(-1, 1), a), test_target()[col].values.reshape(-1, 1)) for a in alphas]
        best_alphas.append(alphas[np.argmin(scores)])
        
    return best_alphas

In [ ]:
@cg.node()
def local_pca_model_submit():
    model = dict()
    for col in tqdm(submit_train_target().columns):
        mask = submit_train_target()[col] == 1
        model[col] = submit_train_features()[mask]
    return model

In [ ]:
@cg.node()
def submit_local_pca_dist():
    dist = pd.DataFrame(index=submit_test_features().index, columns=submit_train_target().columns)
    
    for col in tqdm(submit_train_target().columns):
        if len(local_pca_model_submit()[col]) == 0:
            dist[col] = float('inf')
        else:
            dist[col] = cdist(submit_test_features()[num_columns()].values, local_pca_model_submit()[col][num_columns()]).min(axis=1)
        
    dist[submit_test_features().cp_type == 'ctl_vehicle'] = float('inf')
        
    return dist

In [ ]:
def to_prob(d, a):
    return np.exp(-d * a)

@cg.node()
def submit_prob():
    dist = submit_local_pca_dist()
    return to_prob(dist, get_best_alphas())
submit_prob()

In [ ]:
submit_prob().to_csv('submission.csv')